In [1]:
import os
import time

import refinitiv.data as rd
import pandas as pd
import yfinance as yf
from openai import OpenAI
from pydantic import BaseModel, Field

rd.open_session(os.getenv('REFINITIV_API_KEY'))
try:
    client = OpenAI(api_key=os.getenv("OPENAI_KEY"))
except Exception as e:
    raise e

## Getting the Equities set up

In [116]:
ETFs = {
    'SPDR S&P 500 ETF Trust': 'SPY',
    'SPDR Dow Jones Industrial Average ETF Trust': 'DIA',
    'Invesco NASDAQ 100 ETF': 'QQQM.O',
    'Vanguard FTSE 100 ETF': 'VUKE.L',
    'Xtrackers CAC 40 UCITS ETF': 'XCAC.DE',
    'Global X DAX Germany ETF': 'DAX.O',
    'State Street SPDR EURO STOXX 50 ETF': 'FEZ',
}

In [117]:
tickers = []
for _, etf_ric in ETFs.items():
    print(f"Fetching constituents for ETF: {etf_ric}")
    constituents = rd.get_data(
        universe=[etf_ric],
        fields=['TR.FundHoldingRIC'],
        use_field_names_in_headers=True,
        parameters={'Endnum': '5000'}
    )
    cleaned = constituents[('TR.FundHoldingRIC').upper()].dropna().tolist()
    print(f"Found {len(cleaned)} constituents.")
    tickers.extend(cleaned)

Fetching constituents for ETF: SPY


/Users/cedricmckeever/projects/fin_db/.venv/lib/python3.12/site-packages/refinitiv/data/_access_layer/get_data_func.py:57:FutureWarning: Parameter 'use_field_names_in_headers' is deprecated and will be removed in future library version v2.0.
/Users/cedricmckeever/projects/fin_db/.venv/lib/python3.12/site-packages/refinitiv/data/content/fundamental_and_reference/_definition.py:147:FutureWarning: Parameter 'use_field_names_in_headers' is deprecated and will be removed in future library version v2.0.


Found 503 constituents.
Fetching constituents for ETF: DIA


/Users/cedricmckeever/projects/fin_db/.venv/lib/python3.12/site-packages/refinitiv/data/_access_layer/get_data_func.py:57:FutureWarning: Parameter 'use_field_names_in_headers' is deprecated and will be removed in future library version v2.0.
/Users/cedricmckeever/projects/fin_db/.venv/lib/python3.12/site-packages/refinitiv/data/content/fundamental_and_reference/_definition.py:147:FutureWarning: Parameter 'use_field_names_in_headers' is deprecated and will be removed in future library version v2.0.


Found 30 constituents.
Fetching constituents for ETF: QQQM.O


...

Found 41 constituents.
Fetching constituents for ETF: DAX.O


/Users/cedricmckeever/projects/fin_db/.venv/lib/python3.12/site-packages/refinitiv/data/_access_layer/get_data_func.py:57:FutureWarning: Parameter 'use_field_names_in_headers' is deprecated and will be removed in future library version v2.0.
/Users/cedricmckeever/projects/fin_db/.venv/lib/python3.12/site-packages/refinitiv/data/content/fundamental_and_reference/_definition.py:147:FutureWarning: Parameter 'use_field_names_in_headers' is deprecated and will be removed in future library version v2.0.


Found 40 constituents.
Fetching constituents for ETF: FEZ


/Users/cedricmckeever/projects/fin_db/.venv/lib/python3.12/site-packages/refinitiv/data/_access_layer/get_data_func.py:57:FutureWarning: Parameter 'use_field_names_in_headers' is deprecated and will be removed in future library version v2.0.
/Users/cedricmckeever/projects/fin_db/.venv/lib/python3.12/site-packages/refinitiv/data/content/fundamental_and_reference/_definition.py:147:FutureWarning: Parameter 'use_field_names_in_headers' is deprecated and will be removed in future library version v2.0.


Found 51 constituents.


In [118]:
tickers.extend([   # Add tickers relevant to personal use
    'ALLY.K', 'BABA.K', 'CNI.N', 'DHT.N', 'DSV.CO', 'FND.N', 'FRO.N',
    'GNK.N', 'GOGL.O', 'GXO.N', 'HUBG.O', 'ICGA.DE', 'INSW.K', 'KEX.N',
    'LLYVK.OQ', 'LPGAS', 'LPX.N', 'LSTR.O', 'MANH.O', 'NU.N', 'R.N',
    'SAIA.OQ', 'SBLK.O', 'SIRI.O', 'SNOW.K', 'SOLB.BR', 'STZ.N', 'TSM.N',
    'UBSG.S'
])
unique_tickers = list(set(tickers))
unique_tickers.sort()
print(len(unique_tickers))

747


In [119]:
# Get data
desired = [
    'TR.CommonName',
    'TR.ExchangeCode',
    'TR.HQCountryCode',
    'CF_CURR',
    'TR.GICSSubIndustryCode',
]

equities = rd.get_data(
    universe=unique_tickers,
    fields=desired,
    parameters={'Endnum': '5000'}
)

# Identify empty strings as NaN
equities = equities.replace('', pd.NA)

/Users/cedricmckeever/projects/fin_db/.venv/lib/python3.12/site-packages/refinitiv/data/_access_layer/get_data_func.py:99:UserWarning: 'parameters' is not applicable for fields ['CF_CURR']
/Users/cedricmckeever/projects/fin_db/.venv/lib/python3.12/site-packages/refinitiv/data/_tools/_dataframe.py:192:FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
/Users/cedricmckeever/projects/fin_db/.venv/lib/python3.12/site-packages/refinitiv/data/_tools/_dataframe.py:192:FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


In [120]:
# Drop equities with no name or empty string
equities = equities.dropna(subset=['Company Common Name'])

# Display non-unique common names
non_unique_names = equities['Company Common Name'].value_counts()
problems = non_unique_names[non_unique_names > 1].index.tolist()
print(equities[equities['Company Common Name'].isin(problems)])

    Instrument Company Common Name Exchange Code  \
62     ASML.AS     ASML Holding NV           AEX   
63     ASML.OQ     ASML Holding NV           NSM   
284     FOX.OQ            Fox Corp           NSM   
285    FOXA.OQ            Fox Corp           NSM   
311    GOOG.OQ        Alphabet Inc           NSM   
312   GOOGL.OQ        Alphabet Inc           NSM   
509     NWS.OQ           News Corp           NSM   
510    NWSA.OQ           News Corp           NSM   

    Country ISO Code of Headquarters  GICS Sub-Industry Code CF_CURR  
62                                NL                45301010     EUR  
63                                NL                45301010     USD  
284                               US                50201020     USD  
285                               US                50201020     USD  
311                               US                50203010     USD  
312                               US                50203010     USD  
509                               

In [122]:
# Drop, id rather the ADR
equities= equities[equities['Instrument'] != 'ASML.AS']

# Append to name
class_a = ['FOXA.OQ', 'GOOGL.OQ', 'NWSA.OQ']
for name in class_a:
    equities.loc[equities['Instrument'] == name, 'Company Common Name'] += ' (Class A)'
class_b = ['FOX.OQ', 'NWS.OQ']
for name in class_b:
    equities.loc[equities['Instrument'] == name, 'Company Common Name'] += ' (Class B)'
equities.loc[equities['Instrument'] == 'BRK.B', 'Company Common Name'] += ' (Class C)'
# Also, theres not a lot of rows without GICS code
print(equities[equities['GICS Sub-Industry Code'].isna()]['Company Common Name'])

39                                    Alliance Witan PLC
269                             F&C Investment Trust PLC
310                               Golden Ocean Group Ltd
356                 iShares MSCI China UCITS ETF USD Acc
424                Invesco Government & Agency Portfolio
425    State Street Institutional Government Money Ma...
529                   Polar Capital Technology Trust PLC
558                         Pershing Square Holdings Ltd
622               Scottish Mortgage Investment Trust PLC
Name: Company Common Name, dtype: string


In [123]:
# Drop! Some werent equities -> filtering on this ensures equities
equities = equities.dropna(subset=['GICS Sub-Industry Code'])

# After that step, only 2 NAs left, so might as well drop them too
equities = equities.dropna()

In [124]:
EXCHANGE_CODE_TO_NAME = {
    'NYS': 'New York Stock Exchange',
    'NYQ': 'New York Stock Exchange',
    'NSM': 'NASDAQ',
    'NSQ': 'NASDAQ',
    'NMS': 'NASDAQ',
    'BAT': 'BATS Exchange',
    'LSE': 'London Stock Exchange',
    'BRU': 'Euronext Brussels',
    'PAR': 'Euronext Paris',
    'AEX': 'Euronext Amsterdam',
    'GER': 'Deutsche Börse (Frankfurt)',
    'MCE': 'Bolsa de Madrid',
    'MIL': 'Borsa Italiana (Milan)',
    'CPH': 'Nasdaq Copenhagen',
    'TOR': 'Toronto Stock Exchange',
    'HEX': 'Nasdaq Helsinki',
    'VTX': 'SIX Swiss Exchange',
}

firm_exchange = list(zip(
    equities['Instrument'],
    equities["Company Common Name"],
    equities['Exchange Code'].map(EXCHANGE_CODE_TO_NAME)
))
firm_exchange

[('A.N', 'Agilent Technologies Inc', 'New York Stock Exchange'),
 ('AAF.L', 'Airtel Africa PLC', 'London Stock Exchange'),
 ('AAL.L', 'Anglo American PLC', 'London Stock Exchange'),
 ('AAPL.OQ', 'Apple Inc', 'NASDAQ'),
 ('ABBV.N', 'AbbVie Inc', 'New York Stock Exchange'),
 ('ABF.L', 'Associated British Foods PLC', 'London Stock Exchange'),
 ('ABI.BR', 'Anheuser-Busch Inbev SA', 'Euronext Brussels'),
 ('ABNB.OQ', 'Airbnb Inc', 'NASDAQ'),
 ('ABT.N', 'Abbott Laboratories', 'New York Stock Exchange'),
 ('ACCP.PA', 'Accor SA', 'Euronext Paris'),
 ('ACGL.OQ', 'Arch Capital Group Ltd', 'NASDAQ'),
 ('ACN.N', 'Accenture PLC', 'New York Stock Exchange'),
 ('AD.AS', 'Koninklijke Ahold Delhaize NV', 'Euronext Amsterdam'),
 ('ADBE.OQ', 'Adobe Inc', 'NASDAQ'),
 ('ADI.OQ', 'Analog Devices Inc', 'NASDAQ'),
 ('ADM.N', 'Archer-Daniels-Midland Co', 'New York Stock Exchange'),
 ('ADML.L', 'Admiral Group PLC', 'London Stock Exchange'),
 ('ADP.OQ', 'Automatic Data Processing Inc', 'NASDAQ'),
 ('ADSGn.DE', '

In [125]:
def prep_tuple_list(tuples: list[tuple[str, str, str]]) -> str:
    return ",".join(
        [
        f'("{ric}" - "{company}" - "{exchange}")'
         for ric, company, exchange in tuples
        ]
    )

yfinance_tickers = []

# Limit processed at a time to avoid context length issues
max_batch_size = 25
for i in range(0, len(firm_exchange), max_batch_size):
    batch = firm_exchange[i:min(i+max_batch_size, len(firm_exchange))]
    batch_size = len(batch)
    user_content = prep_tuple_list(batch)

    class IntendedResult(BaseModel):
        tickers: list[str] | None = Field(min_length=batch_size, max_length=batch_size)

    system_prompt = (
        "You will be provided a list of tuples representing a refinitiv RIC, "
        "company, and exchange as such: ('RIC' - 'Company Name' - 'Exchange Name'), "
        "('RIC' - 'Company Name' - 'Exchange Name'), ... "
        "\nYou must return a vector of corresponding yahoo finance tickers."
    )

    response = client.chat.completions.parse(
        model="gpt-5-nano",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_content},
        ],
        response_format=IntendedResult,  # Enforce structured output
    )

    parsed = response.choices[0].message.parsed
    if parsed and parsed.tickers:
        yfinance_tickers.extend(parsed.tickers)
        print(
            f"Added {len(parsed.tickers)} tickers"
            f"({len(yfinance_tickers)/len(firm_exchange):.2%})"
        )
    else:
        print("No tickers returned for this batch.")

Added 25 tickers(3.42%)
Added 25 tickers(6.83%)
Added 25 tickers(10.25%)
Added 25 tickers(13.66%)
Added 25 tickers(17.08%)
Added 25 tickers(20.49%)
Added 25 tickers(23.91%)
Added 25 tickers(27.32%)
Added 25 tickers(30.74%)
Added 25 tickers(34.15%)
Added 25 tickers(37.57%)
Added 25 tickers(40.98%)
Added 25 tickers(44.40%)
Added 25 tickers(47.81%)
Added 25 tickers(51.23%)
Added 25 tickers(54.64%)
Added 25 tickers(58.06%)
Added 25 tickers(61.48%)
Added 25 tickers(64.89%)
Added 25 tickers(68.31%)
Added 25 tickers(71.72%)
Added 25 tickers(75.14%)
Added 25 tickers(78.55%)
Added 25 tickers(81.97%)
Added 25 tickers(85.38%)
Added 25 tickers(88.80%)
Added 25 tickers(92.21%)
Added 25 tickers(95.63%)
Added 25 tickers(99.04%)
Added 7 tickers(100.00%)


In [126]:
equities['yfinance_ticker'] = yfinance_tickers

In [127]:
lseg_prices = rd.get_history(
    universe=equities['Instrument'].tolist(),
    fields=['TR.PriceClose'],
    start='2026-02-01',
    end='2026-02-05',
)

/Users/cedricmckeever/projects/fin_db/.venv/lib/python3.12/site-packages/refinitiv/data/_tools/_dataframe.py:177:FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


In [128]:
# Make numeric
equities['lseg_valid'] = pd.to_numeric(
    equities['Instrument'].map(lseg_prices.transpose()['2026-02-03']),
    errors='coerce'
)

In [ ]:
# Retrieve price from yfinance to compare
yfin = equities['yfinance_ticker'].to_list()

retrievals = []

# Limit tickers to stay safe
size = 25
for i in range(0, len(yfin), size):
    batch = yfin[i:min(i+size, len(yfin))]
    prices = yf.download(
        tickers=batch,
        start='2026-02-01',
        end='2026-02-05',
    )
    close = prices.xs('Close', axis=1, level=0)
    retrievals.append(close)
    time.sleep(1)  # Be nice to Yahoo's servers

yfin_prices = pd.concat(retrievals, axis=1)

[*********************100%***********************]  25 of 25 completed
[**********************88%*****************      ]  22 of 25 completed$AIRP.PA: possibly delisted; no price data found  (1d 2026-02-01 -> 2026-02-05)
[*********************100%***********************]  25 of 25 completed

1 Failed download:
['AIRP.PA']: possibly delisted; no price data found  (1d 2026-02-01 -> 2026-02-05)
[*****************     36%                       ]  9 of 25 completed$AUTOA.L: possibly delisted; no timezone found
[*******************   40%                       ]  10 of 25 completed$AXA.PA: possibly delisted; no timezone found
[*********************100%***********************]  25 of 25 completed

2 Failed downloads:
['AUTOA.L', 'AXA.PA']: possibly delisted; no timezone found
[*******************   40%                       ]  10 of 25 completed$BLL: possibly delisted; no timezone found
[**********************64%******                 ]  16 of 25 completed$BF.B: possibly delisted; no price dat

In [130]:
equities['yfin_valid'] = equities['yfinance_ticker'].map(
    yfin_prices.transpose()['2026-02-03']
)
equities['distance'] = (equities['lseg_valid'] - equities['yfin_valid']).abs() / equities['lseg_valid']
equities[equities['distance'] > 0.005]

,Instrument,Company Common Name,Exchange Code,Country ISO Code of Headquarters,GICS Sub-Industry Code,CF_CURR,yfinance_ticker,lseg_valid,yfin_valid,distance
140,CFG.N,Citizens Financial Group Inc,NYS,US,40101015,USD,CFG,65.97,65.510002,0.006973


In [5]:
equities = pd.read_csv('equities_saved.csv')

In [6]:
# Attempt to build ticker for those that failed
# =============================================
problems = equities['yfin_valid'].isna()
print(f"Found {problems.sum()} problematic tickers.")

print(
    equities['Instrument']
    .apply(lambda x: x.split('.')[-1] if '.' in x else None)
    .unique()
)

RIC_TO_YAHOO_SUFFIX = {
    '.N': '',       # NYSE
    '.O': '',       # NASDAQ
    '.OQ': '',      # NASDAQ
    '.K': '',       # NYSE (alternate)
    '.A': '',       # NYSE Amex
    '.L': '.L',     # London Stock Exchange
    '.DE': '.DE',   # Frankfurt (Xetra)
    '.AS': '.AS',   # Amsterdam
    '.PA': '.PA',   # Paris
    '.BR': '.BR',   # Brussels
    '.MC': '.MC',   # Madrid
    '.MI': '.MI',   # Milan
    '.CO': '.CO',   # Copenhagen
    '.HE': '.HE',   # Helsinki
    '.S': '.ST',    # Stockholm
    '.SW': '.SW',   # Swiss Exchange
    '.TO': '.TO',   # Toronto
}

equities.loc[problems, 'yfinance_ticker'] = (
    equities.loc[problems, 'Instrument']
    .apply(
        lambda x: x.split('.')[0] + RIC_TO_YAHOO_SUFFIX.get(
                ('.' + x.split('.')[-1]), ''
            )
    )
)

Found 53 problematic tickers.
['N' 'L' 'OQ' 'BR' 'PA' 'AS' 'DE' 'K' 'MC' 'Z' 'MI' 'CO' 'TO' 'O' 'HE' 'S']


In [7]:
# Re-fetch yahpoo prices for corrected tickers
# ==========================================
problems = equities['yfin_valid'].isna()
print(f"Re-fetching {problems.sum()} problematic tickers.")

yfin = equities.loc[problems, 'yfinance_ticker'].to_list()
retrievals = [] 
# Limit tickers to stay safe
size = 25
for i in range(0, len(yfin), size):
    batch = yfin[i:min(i+size, len(yfin))]
    prices = yf.download(
        tickers=batch,
        start='2026-02-01',
        end='2026-02-05',
    )
    close = prices.xs('Close', axis=1, level=0)
    close = close.transpose()['2026-02-03']
    retrievals.append(close)
    time.sleep(1)  # Be nice to Yahoo's servers

yfin_prices = pd.concat(retrievals, axis=0)
equities.loc[problems, 'yfin_valid'] = equities.loc[problems, 'yfinance_ticker'].map(
    yfin_prices
)
equities['distance'] = (equities['lseg_valid'] - equities['yfin_valid']).abs() / equities['lseg_valid']
equities[equities['distance'] > 0.005]
print(equities['yfin_valid'].isna().sum())

Re-fetching 53 problematic tickers.


$HNKG_P.DE: possibly delisted; no price data found  (1d 2026-02-01 -> 2026-02-05)
[                       0%                       ]$BKGH.L: possibly delisted; no price data found  (1d 2026-02-01 -> 2026-02-05)
[********              16%                       ]  4 of 25 completed$DAST.PA: possibly delisted; no price data found  (1d 2026-02-01 -> 2026-02-05)
[**********            20%                       ]  5 of 25 completed$AIRP.PA: possibly delisted; no price data found  (1d 2026-02-01 -> 2026-02-05)
$AXAF.PA: possibly delisted; no price data found  (1d 2026-02-01 -> 2026-02-05)
[************          24%                       ]  6 of 25 completed$EUFI.PA: possibly delisted; no price data found  (1d 2026-02-01 -> 2026-02-05)
[*************         28%                       ]  7 of 25 completed$FOUG.PA: possibly delisted; no price data found  (1d 2026-02-01 -> 2026-02-05)
[***************       32%                       ]  8 of 25 completed$DB1GN.DE: possibly delisted; no price data 

44


Fixed some, but not all :(

In [8]:
# Remaing problems
remaining_problems = equities['yfin_valid'].isna()

# Trying a fix with copilot
SPECIAL_RIC_TO_YAHOO = {
    'AIRP.PA': 'AI.PA',
    'AUTOA.L': 'AUTO.L',
    'AXAF.PA': 'CS.PA',
    'BFb.N': 'BF-B',
    'BKGH.L': 'BKG.L',
    'BNRGn.DE': 'BNRG.DE',
    'BOUY.PA': 'EN.PA',
    'BT.L': 'BT-A.L',
    'CAGR.PA': 'CAF.PA',
    'DANO.PA': 'BN.PA',
    'DAST.PA': 'DSY.PA',
    'DB1Gn.DE': 'DB1.DE',
    'DHLn.DE': 'DHL.DE',
    'ESLX.PA': 'ESI.PA',
    'EUFI.PA': 'URW.AS',
    'FMEG.DE': 'FME.DE',
    'FOUG.PA': 'FOUG.PA',
    'FREG.DE': 'FRE.DE',
    'G1AG.DE': 'G1A.DE',
    'G24n.DE': 'G24.DE',
    'HNKG_p.DE': 'HNR1.DE',
    'HNRGn.DE': 'HNR.DE',
    'HRMS.PA': 'RMS.PA',
    'LEGD.PA': 'LI.PA',
    'LVMH.PA': 'MC.PA',
    'MICP.PA': 'ML.PA',
    'MRCG.DE': 'MRK.DE',
    'MRON.L': 'MRO.L',
    'NDAFI.HE': 'NESTE.HE',
    'ORAN.PA': 'ORA.PA',
    'PERP.PA': 'RI.PA',
    'PRTP.PA': 'UG.PA',
    'PSHG_p.DE': 'PAH3.DE',
    'PUBP.PA': 'PUB.PA',
    'RENA.PA': 'RNO.PA',
    'SASY.PA': 'SAN.PA',
    'SCHN.PA': 'SGO.PA',
    'SGOB.PA': 'GLE.PA',
    'SJP.L': 'SJP.L',
    'SOGN.PA': 'SW.PA',
    'SY1G.DE': 'SY1.DE',
    'TCFP.PA': 'HO.PA',
    'TTEF.PA': 'FP.PA',
    'VOWG_p.DE': 'VOW3.DE',
}

# Apply fixes
equities.loc[remaining_problems, 'yfinance_ticker'] = (
    equities.loc[remaining_problems, 'Instrument'].replace(SPECIAL_RIC_TO_YAHOO)
)

# Again re-fetch yahpoo prices for corrected tickers
# ==========================================
yfin = equities.loc[remaining_problems, 'yfinance_ticker'].to_list()
retrievals = [] 
# Limit tickers to stay safe
size = 25
for i in range(0, len(yfin), size):
    batch = yfin[i:min(i+size, len(yfin))]
    prices = yf.download(
        tickers=batch,
        start='2026-02-01',
        end='2026-02-05',
    )
    close = prices.xs('Close', axis=1, level=0)
    close = close.transpose()['2026-02-03']
    retrievals.append(close)
    time.sleep(1)  # Be nice to Yahoo's servers

yfin_prices = pd.concat(retrievals, axis=0)
equities.loc[remaining_problems, 'yfin_valid'] = equities.loc[remaining_problems, 'yfinance_ticker'].map(
    yfin_prices
)
equities['distance'] = (equities['lseg_valid'] - equities['yfin_valid']).abs() / equities['lseg_valid']
print(equities[equities['distance'] > 0.005])
print(equities['yfin_valid'].isna().sum())


$FOUG.PA: possibly delisted; no price data found  (1d 2026-02-01 -> 2026-02-05)
[***************       32%                       ]  8 of 25 completed$HNR.DE: possibly delisted; no timezone found
[********************* 44%                       ]  11 of 25 completed$URW.AS: possibly delisted; no timezone found
$BNRG.DE: possibly delisted; no timezone found
[**********************92%*******************    ]  23 of 25 completed$ESI.PA: possibly delisted; no timezone found
[*********************100%***********************]  25 of 25 completed

5 Failed downloads:
['FOUG.PA']: possibly delisted; no price data found  (1d 2026-02-01 -> 2026-02-05)
['HNR.DE', 'URW.AS', 'BNRG.DE', 'ESI.PA']: possibly delisted; no timezone found
$SJP.L: possibly delisted; no timezone found
[**********************58%***                    ]  11 of 19 completed$FP.PA: possibly delisted; no timezone found
[**********************89%******************     ]  17 of 19 completed$UG.PA: possibly delisted; no timezone fo

    Instrument           Company Common Name Exchange Code  \
120    CAGR.PA            Credit Agricole SA           PAR   
138      CFG.N  Citizens Financial Group Inc           NYS   
326  HNKG_p.DE           Henkel AG & Co KGaA           GER   
403    LEGD.PA                    Legrand SA           PAR   
477   NDAFI.HE               Nordea Bank Abp           HEX   
590    SCHN.PA         Schneider Electric SE           PAR   
595    SGOB.PA  Compagnie de Saint Gobain SA           PAR   
614    SOGN.PA           Societe Generale SA           PAR   

    Country ISO Code of Headquarters  GICS Sub-Industry Code CF_CURR  \
120                               FR                40101010     EUR   
138                               US                40101015     USD   
326                               DE                30301010     EUR   
403                               FR                20104010     EUR   
477                               FI                40101010     EUR   
590      

In [9]:
# Fix the remaining manually
empties = equities['yfin_valid'].isna()
suspects = equities['distance'] > 0.005
equities[empties | suspects]

,Instrument,Company Common Name,Exchange Code,Country ISO Code of Headquarters,GICS Sub-Industry Code,CF_CURR,yfinance_ticker,lseg_valid,yfin_valid,distance
104,BNRGn.DE,Brenntag SE,GER,DE,20107010,EUR,BNRG.DE,51.820,NaN,NaN
120,CAGR.PA,Credit Agricole SA,PAR,FR,40101010,EUR,CAF.PA,18.745,119.500000,5.375033
138,CFG.N,Citizens Financial Group Inc,NYS,US,40101015,USD,CFG,65.970,65.510002,0.006973
247,ESLX.PA,EssilorLuxottica SA,PAR,FR,35101020,EUR,ESI.PA,255.900,NaN,NaN
251,EUFI.PA,Eurofins Scientific SE,PAR,LU,35203010,EUR,URW.AS,67.880,NaN,NaN
278,FOUG.PA,Eiffage SA,PAR,FR,20103010,EUR,FOUG.PA,127.600,NaN,NaN
326,HNKG_p.DE,Henkel AG & Co KGaA,GER,DE,30301010,EUR,HNR1.DE,75.540,242.800003,2.214191
327,HNRGn.DE,Hannover Rueck SE,GER,DE,40301050,EUR,HNR.DE,242.800,NaN,NaN
403,LEGD.PA,Legrand SA,PAR,FR,20104010,EUR,LI.PA,137.200,31.959999,0.767055
477,NDAFI.HE,Nordea Bank Abp,HEX,FI,40101010,EUR,NESTE.HE,16.905,21.610001,0.278320


In [ ]:
MANUAL_FIX = {
    'BALL.N': 'BALL',
    'BNRGn.DE': 'BNR.DE',
    'CAGR.PA': 'ACA.PA',
    'CFG.N': 'CFG',
    'DOC.N': 'DOC',
    'ESLX.PA': 'EL.PA',
    'EUFI.PA': 'ERF.PA',
    'FOUG.PA': 'FGR.PA',
    'HNKG_p.DE': 'HEN3.DE',
    'HNRGn.DE': 'HNR1.DE',
    'LEGD.PA': 'LR.PA',
    'LLYVK.OQ': 'LLYVK',
    'NDAFI.HE': 'NDA-FI.HE',
    'PRTP.PA': 'KER.PA',
    'SCHN.PA': 'SU.PA',
    'SGOB.PA': 'SGO.PA',
    'SJP.L': 'STJ.L',
    'SOGN.PA': 'GLE.PA',
    'SW.N': 'SW',
    'TTEF.PA': 'TTE.PA',
    'XYZ.N': 'XYZ'
}

equities.loc[empties | suspects, 'yfinance_ticker'] = (
    equities.loc[empties | suspects, 'Instrument'].replace(MANUAL_FIX)
)

Last check ?

In [11]:
# Retrieve price from yfinance to compare
yfin = equities['yfinance_ticker'].to_list()

retrievals = []

# Limit tickers to stay safe
size = 25
for i in range(0, len(yfin), size):
    batch = yfin[i:min(i+size, len(yfin))]
    prices = yf.download(
        tickers=batch,
        start='2026-02-01',
        end='2026-02-05',
    )
    close = prices.xs('Close', axis=1, level=0)
    retrievals.append(close)
    time.sleep(1)  # Be nice to Yahoo's servers

yfin_prices = pd.concat(retrievals, axis=1)

equities['yfin_valid'] = equities['yfinance_ticker'].map(
    yfin_prices.transpose()['2026-02-03']
)
equities['distance'] = (equities['lseg_valid'] - equities['yfin_valid']).abs() / equities['lseg_valid']

[*********************100%***********************]  25 of 25 completed
[*********************100%***********************]  25 of 25 completed
[*********************100%***********************]  25 of 25 completed
[*********************100%***********************]  25 of 25 completed
[*********************100%***********************]  25 of 25 completed
[*********************100%***********************]  25 of 25 completed
[*********************100%***********************]  25 of 25 completed
[*********************100%***********************]  25 of 25 completed
[*********************100%***********************]  25 of 25 completed
[*********************100%***********************]  25 of 25 completed
[********************* 44%                       ]  11 of 25 completed$EVR.L: possibly delisted; no price data found  (1d 2026-02-01 -> 2026-02-05) (Yahoo error = "No data found, symbol may be delisted")
[*********************100%***********************]  25 of 25 completed

1 Failed downl

In [12]:
# Check last stragglers
empties = equities['yfin_valid'].isna()
suspects = equities['distance'] > 0.005
equities[empties | suspects]

,Instrument,Company Common Name,Exchange Code,Country ISO Code of Headquarters,GICS Sub-Industry Code,CF_CURR,yfinance_ticker,lseg_valid,yfin_valid,distance
138,CFG.N,Citizens Financial Group Inc,NYS,US,40101015,USD,CFG,65.97,65.510002,0.006973
252,EVRE.L,EVRAZ plc,LSE,GB,15104050,GBp,EVR.L,NaN,NaN,NaN
598,SHLG.DE,Siemens Healthineers AG,GER,DE,35101010,EUR,SHL.DE,42.15,41.160564,0.023474
661,TSCO.OQ,Tractor Supply Co,NSM,US,25504040,USD,TSCO,53.47,NaN,NaN
689,VLO.N,Valero Energy Corp,NYS,US,10102030,USD,VLO,192.27,191.101242,0.006079


Small inconstitenties, but data is okay expect one delisted!

In [13]:
# Drop because unlisted
equities = equities[equities['Instrument'] != 'EVRE.L']

equities

,Instrument,Company Common Name,Exchange Code,Country ISO Code of Headquarters,GICS Sub-Industry Code,CF_CURR,yfinance_ticker,lseg_valid,yfin_valid,distance
0,A.N,Agilent Technologies Inc,NYS,US,35203010,USD,A,132.14,132.139999,4.618976e-09
1,AAF.L,Airtel Africa PLC,LSE,GB,50102010,GBp,AAF.L,324.00,324.000000,0.000000e+00
2,AAL.L,Anglo American PLC,LSE,GB,15104020,GBp,AAL.L,3700.00,3700.000000,0.000000e+00
3,AAPL.OQ,Apple Inc,NSM,US,45202030,USD,AAPL,269.48,269.480011,4.076862e-08
4,ABBV.N,AbbVie Inc,NYS,US,35201010,USD,ABBV,225.66,225.660004,1.622844e-08
...,...,...,...,...,...,...,...,...,...,...
727,ZALG.DE,Zalando SE,GER,DE,25504010,EUR,ZAL.DE,21.48,21.480000,2.131116e-08
728,ZBH.N,Zimmer Biomet Holdings Inc,NYS,US,35101010,USD,ZBH,86.18,86.180000,3.541144e-09
729,ZBRA.OQ,Zebra Technologies Corp,NSM,US,45203010,USD,ZBRA,233.16,233.160004,1.570642e-08
730,ZS.OQ,Zscaler Inc,NSM,US,45103020,USD,ZS,188.05,188.050003,1.622844e-08


In [ ]:
# Last fields I want from rd (forgot)
fetch = rd.get_data(
    universe=list(equities['Instrument']),
    fields='TR.ISIN',
    parameters={'Endnum': '5000'}
).replace('', pd.NA)

# Identify empty strings as NaN
fetch_map = fetch.set_index('Instrument').to_dict()['ISIN']
equities['isin'] = equities['Instrument'].map(fetch_map)

In [ ]:
# Map the etoro tickers I do have
etoro = pd.read_csv('etoro_ticker_mapper.csv')
etoro_mapper = dict(zip(etoro['LSEG'], etoro['ETORO']))
equities['etoro_ticker'] = equities['Instrument'].map(etoro_mapper)

In [40]:
# Rename columns
rename_map = {
    'Instrument': 'ric',
    'Company Common Name': 'name',
    'Exchange Code': 'exchange',
    'Country ISO Code of Headquarters': 'country_hq',
    'GICS Sub-Industry Code': 'gics_code',
    'CF_CURR': 'currency',
    'ISIN': 'isin'
}

equities = equities.rename(columns=rename_map)

# Reorder columns and keep only relevant ones
equities = equities[[
    'name', 'isin', 'yfinance_ticker', 'etoro_ticker', 'ric',
    'exchange', 'country_hq', 'currency', 'gics_code'
]]

In [43]:
equities.to_csv('equities_final.csv', index=False)